In [6]:
import os
import boto3
import re
import sagemaker
import numpy as np
import pandas as pd
import sklearn.model_selection
from sklearn.model_selection import train_test_split

# Import, subject to change once imported to aws
df = pd.read_csv("user_data.csv")

# Data cleaning most likely needed here but format unknow so blank for now

# Splits data into train and test frames
train, test = train_test_split(df, test_size=.3)

# Finds unique users and pages
def get_unique_count(df):
    unique = []
    for x in df.index:
        unique.append(re.search("pagePath(.*)", df.hits[x]))
        unique[len(unique)-1] = re.search("/google(.*)", str(unique[len(unique)-1]))
    unique = list(set(unique))
    unique_pages = len(unique)
    return len(df['fullVisitorId'].value_counts()), unique_pages

# Stores number of users/pages for later usage
n_user, n_item = get_unique_count(train)
%store n_user
%store n_item

# Making bucket for transport of data
session = boto3.session.Session()
region = session.region_name
sage_session = sagemaker.Session()
bucket = sage_session.default_bucket()

train_path = os.path.join("Recommendation Engine/data/", "train.npy")
test_path = os.path.join("Recommendation Engine/data/", "test.npy")

# Creates local dataset to represent test and train from code.
np.save('train', train.values)
np.save('test', test.values)

# Upload data to model.
sage_session.upload_data('test.npy', bucket=bucket, key_prefix='data')
sage_session.upload_data('train.npy', bucket=bucket, key_prefix='data')


Stored 'n_user' (int)
Stored 'n_item' (int)


's3://sagemaker-us-east-1-350614259618/data/train.npy'